# I. Xử lý dữ liệu
## Trích xuất đặc trưng của file PE. Bao gồm 69 đặc trưng:
## Loại bỏ đặc trưng packer_type do dữ liệu không phải dạng số (vd: VC8_Microsoft_Corporation)

IMAGE_DOS_HEADER (6)
- e_cblp
- e_cp
- e_cparhdr
- e_maxalloc
- e_sp
- e_lfanew

FILE_HEADER (17)
- NumberOfSections
- CreationYear
- FH_char0
- FH_char1
- FH_char2
- FH_char3
- FH_char4
- FH_char5
- FH_char6
- FH_char7
- FH_char8
- FH_char9
- FH_char10
- FH_char11
- FH_char12
- FH_char13
- FH_char14

OPTIONAL_HEADER (37)
- MajorLinkerVersion
- MinorLinkerVersion
- SizeOfCode
- SizeOfInitializedData
- SizeOfUninitializedData
- AddressOfEntryPoint
- BaseOfCode
- BaseOfData
- ImageBase
- SectionAlignment
- FileAlignment
- MajorOperatingSystemVersion
- MinorOperatingSystemVersion
- MajorImageVersion
- MinorImageVersion
- MajorSubsystemVersion
- MinorSubsystemVersion
- SizeOfImage
- SizeOfHeaders
- CheckSum
- Subsystem
- OH_DLLchar0
- OH_DLLchar1
- OH_DLLchar2
- OH_DLLchar3
- OH_DLLchar4
- OH_DLLchar5
- OH_DLLchar6
- OH_DLLchar7
- OH_DLLchar8
- OH_DLLchar9
- OH_DLLchar10
- SizeOfStackReserve
- SizeOfStackCommit
- SizeOfHeapReserve
- SizeOfHeapCommit
- LoaderFlags

Derived Header (8)
- sus_sections
- non_sus_sections
- packer
- E_text
- E_data
- filesize
- E_file
- fileinfo

In [1]:
# II. Dataset


In [2]:
!pip install pandas
!pip install pefile
!pip install yara

In [3]:
import pickle as pkl
import pandas as pd
import pefile
import pefile as pe
import sys
import numpy as np
import pandas as pd
import math, csv, os
import yara

In [4]:
class pe_features():

    IMAGE_DOS_HEADER = [
                        "e_cblp",\
                        "e_cp", \
                        "e_cparhdr",\
                        "e_maxalloc",\
                        "e_sp",\
                        "e_lfanew"]

    FILE_HEADER= ["NumberOfSections","CreationYear"] + [ "FH_char" + str(i) for i in range(15)]
                

    OPTIONAL_HEADER1 = [
                        "MajorLinkerVersion",\
                        "MinorLinkerVersion",\
                        "SizeOfCode",\
                        "SizeOfInitializedData",\
                        "SizeOfUninitializedData",\
                        "AddressOfEntryPoint",\
                        "BaseOfCode",\
                        "BaseOfData",\
                        "ImageBase",\
                        "SectionAlignment",\
                        "FileAlignment",\
                        "MajorOperatingSystemVersion",\
                        "MinorOperatingSystemVersion",\
                        "MajorImageVersion",\
                        "MinorImageVersion",\
                        "MajorSubsystemVersion",\
                        "MinorSubsystemVersion",\
                        "SizeOfImage",\
                        "SizeOfHeaders",\
                        "CheckSum",\
                        "Subsystem"] 
    OPTIONAL_HEADER_DLL_char = [ "OH_DLLchar" + str(i) for i in range(11)]                   
                            
    OPTIONAL_HEADER2 = [
                        "SizeOfStackReserve",\
                        "SizeOfStackCommit",\
                        "SizeOfHeapReserve",\
                        "SizeOfHeapCommit",\
                        "LoaderFlags"]  # boolean check for zero or not
    
    OPTIONAL_HEADER = OPTIONAL_HEADER1 + OPTIONAL_HEADER_DLL_char + OPTIONAL_HEADER2
    Derived_header = ["sus_sections","non_sus_sections", "packer","E_text","E_data","filesize","E_file","fileinfo"]
    # Derived_header = ["sus_sections","non_sus_sections", "packer","packer_type","E_text","E_data","filesize","E_file","fileinfo"]
    
    def __init__(self, source, output, label):
        self.source = source
        self.output = output
        self.type = label
        self.rules= yara.compile('peid.yar')
        
    def file_creation_year(self,seconds):
        tmp = 1970 + ((int(seconds) / 86400) / 365)
        return int(tmp in range (1980,2016)) 


    def FILE_HEADER_Char_boolean_set(self,pe):
        tmp = [pe.FILE_HEADER.IMAGE_FILE_RELOCS_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_EXECUTABLE_IMAGE,\
            pe.FILE_HEADER.IMAGE_FILE_LINE_NUMS_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_LOCAL_SYMS_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_AGGRESIVE_WS_TRIM,\
            pe.FILE_HEADER.IMAGE_FILE_LARGE_ADDRESS_AWARE,\
            pe.FILE_HEADER.IMAGE_FILE_BYTES_REVERSED_LO,\
            pe.FILE_HEADER.IMAGE_FILE_32BIT_MACHINE,\
            pe.FILE_HEADER.IMAGE_FILE_DEBUG_STRIPPED,\
            pe.FILE_HEADER.IMAGE_FILE_REMOVABLE_RUN_FROM_SWAP,\
            pe.FILE_HEADER.IMAGE_FILE_NET_RUN_FROM_SWAP,\
            pe.FILE_HEADER.IMAGE_FILE_SYSTEM,\
            pe.FILE_HEADER.IMAGE_FILE_DLL,\
            pe.FILE_HEADER.IMAGE_FILE_UP_SYSTEM_ONLY,\
            pe.FILE_HEADER.IMAGE_FILE_BYTES_REVERSED_HI
            ]
        return [int(s) for s in tmp]

    def OPTIONAL_HEADER_DLLChar(self,pe):
        tmp = [
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_DYNAMIC_BASE,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_FORCE_INTEGRITY,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NX_COMPAT,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_ISOLATION,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_SEH,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_BIND,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_WDM_DRIVER,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_TERMINAL_SERVER_AWARE,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_HIGH_ENTROPY_VA,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_APPCONTAINER,\
            pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_GUARD_CF
            ]
        return [int(s) for s in tmp]
    def Optional_header_ImageBase(self,ImageBase):
        result= 0
        if ImageBase % (64 * 1024) == 0 and ImageBase in [268435456,65536,4194304]:
            result = 1
        return result
    def Optional_header_SectionAlignment(self,SectionAlignment,FileAlignment):
        """This is boolean function and will return 0 or 1 based on condidtions
        that it SectionAlignment must be greater than or equal to FileAlignment
        """
        return int(SectionAlignment >= FileAlignment)
    def Optional_header_FileAlignment(self,SectionAlignment,FileAlignment):
        result =0
        if SectionAlignment >= 512:
            if FileAlignment % 2 == 0 and FileAlignment in range(512,65537):
                result =1
        else: 
            if FileAlignment == SectionAlignment:
                result = 1
        return result

    def Optional_header_SizeOfImage(self,SizeOfImage,SectionAlignment):
        return int(SizeOfImage % SectionAlignment == 0)

    def Optional_header_SizeOfHeaders(self,SizeOfHeaders,FileAlignment):
        return int(SizeOfHeaders % FileAlignment == 0 )


    def extract_dos_header(self,pe):
        IMAGE_DOS_HEADER_data = [ 0 for i in range(6)]
        try:
            IMAGE_DOS_HEADER_data = [
                                pe.DOS_HEADER.e_cblp,\
                                pe.DOS_HEADER.e_cp, \
                                pe.DOS_HEADER.e_cparhdr,\
                                pe.DOS_HEADER.e_maxalloc,\
                                pe.DOS_HEADER.e_sp,\
                                pe.DOS_HEADER.e_lfanew]
        except Exception as e:
            # print(e)
            pass
        return IMAGE_DOS_HEADER_data

    def extract_file_header(self,pe):	
        FILE_HEADER_data = [ 0 for i in range(3)]
        FILE_HEADER_char =  []
        try:
            FILE_HEADER_data = [ 
                    pe.FILE_HEADER.NumberOfSections, \
                    self.file_creation_year(pe.FILE_HEADER.TimeDateStamp)]
            FILE_HEADER_char = self.FILE_HEADER_Char_boolean_set(pe)
        except Exception as e:
            # print(e)
            pass
        return FILE_HEADER_data + FILE_HEADER_char

    def extract_optional_header(self,pe):
        OPTIONAL_HEADER_data = [ 0 for i in range(21)]
        DLL_char =[]
        OPTIONAL_HEADER_data2 = [ 0 for i in range(6)]

        try:
            OPTIONAL_HEADER_data = [
                pe.OPTIONAL_HEADER.MajorLinkerVersion,\
                pe.OPTIONAL_HEADER.MinorLinkerVersion,\
                pe.OPTIONAL_HEADER.SizeOfCode,\
                pe.OPTIONAL_HEADER.SizeOfInitializedData,\
                pe.OPTIONAL_HEADER.SizeOfUninitializedData,\
                pe.OPTIONAL_HEADER.AddressOfEntryPoint,\
                pe.OPTIONAL_HEADER.BaseOfCode,\
                pe.OPTIONAL_HEADER.BaseOfData,\
                #Check the ImageBase for the condition
                self.Optional_header_ImageBase(pe.OPTIONAL_HEADER.ImageBase),\
                # Checking for SectionAlignment condition
                self.Optional_header_SectionAlignment(pe.OPTIONAL_HEADER.SectionAlignment,pe.OPTIONAL_HEADER.FileAlignment),\
                #Checking for FileAlignment condition
                self.Optional_header_FileAlignment(pe.OPTIONAL_HEADER.SectionAlignment,pe.OPTIONAL_HEADER.FileAlignment),\
                pe.OPTIONAL_HEADER.MajorOperatingSystemVersion,\
                pe.OPTIONAL_HEADER.MinorOperatingSystemVersion,\
                pe.OPTIONAL_HEADER.MajorImageVersion,\
                pe.OPTIONAL_HEADER.MinorImageVersion,\
                pe.OPTIONAL_HEADER.MajorSubsystemVersion,\
                pe.OPTIONAL_HEADER.MinorSubsystemVersion,\
                #Checking size of Image
                self.Optional_header_SizeOfImage(pe.OPTIONAL_HEADER.SizeOfImage,pe.OPTIONAL_HEADER.SectionAlignment),\
                #Checking for size of headers
                self.Optional_header_SizeOfHeaders(pe.OPTIONAL_HEADER.SizeOfHeaders,pe.OPTIONAL_HEADER.FileAlignment),\
                pe.OPTIONAL_HEADER.CheckSum,\
                pe.OPTIONAL_HEADER.Subsystem]

            DLL_char = self.OPTIONAL_HEADER_DLLChar(pe)

            OPTIONAL_HEADER_data2= [                
                pe.OPTIONAL_HEADER.SizeOfStackReserve,\
                pe.OPTIONAL_HEADER.SizeOfStackCommit,\
                pe.OPTIONAL_HEADER.SizeOfHeapReserve,\
                pe.OPTIONAL_HEADER.SizeOfHeapCommit,\
                int(pe.OPTIONAL_HEADER.LoaderFlags == 0) ]
        except Exception as e:
            # print(e)
            pass
        return OPTIONAL_HEADER_data + DLL_char + OPTIONAL_HEADER_data2

    def get_count_suspicious_sections(self,pe):
        result=[]
        tmp =[]
        benign_sections = set(['.text','.data','.rdata','.idata','.edata','.rsrc','.bss','.crt','.tls'])
        for section in pe.sections:
            tmp.append(section.Name.split(b'\x00')[0])
        non_sus_sections = len(set(tmp).intersection(benign_sections))
        result=[len(tmp) - non_sus_sections, non_sus_sections]
        return result


    def check_packer(self,filepath):
        result=[]
        matches = self.rules.match(filepath)
        if matches == []:
            result.append([0,"NoPacker"])
        else:
            #result.append([1,matches['main'][0]['rule']])
            result.append([1,matches[0]])
        return result

    def get_text_data_entropy(self,pe):
        result=[0.0,0.0]
        for section in pe.sections:
            s_name = section.Name.split(b'\x00')[0]
            if s_name == ".text":
                result[0]= section.get_entropy()
            elif s_name == ".data":
                result[1]= section.get_entropy()
            else:
                pass
        return result  

    def get_file_bytes_size(self,filepath):
        f = open(filepath, "rb")
        data = f.read()
        # byteArr = [ord(c) for c in f.read()]
        #f.close()
        fileSize = len(data)
        return data, fileSize

    def cal_byteFrequency(self,byteArr,fileSize):
        freqList = []
        for b in range(256):
            ctr = 0
            for byte in byteArr:
                if byte == b:
                    ctr += 1
            freqList.append(float(ctr) / fileSize)
        return freqList

    def get_file_entropy(self,filepath):
        byteArr, fileSize = self.get_file_bytes_size(filepath)
        freqList = self.cal_byteFrequency(byteArr,fileSize)
        # Shannon entropy
        ent = 0.0
        for freq in freqList:
            if freq > 0:
                ent +=  - freq * math.log(freq, 2)

            #ent = -ent
        return [fileSize,ent]
    
    def get_fileinfo(self,pe):
        result=[]
        try:
            FileVersion    = pe.FileInfo[0].StringTable[0].entries['FileVersion']
            ProductVersion = pe.FileInfo[0].StringTable[0].entries['ProductVersion']
            ProductName =    pe.FileInfo[0].StringTable[0].entries['ProductName']
            CompanyName = pe.FileInfo[0].StringTable[0].entries['CompanyName']
    #getting Lower and 
            FileVersionLS    = pe.VS_FIXEDFILEINFO.FileVersionLS
            FileVersionMS    = pe.VS_FIXEDFILEINFO.FileVersionMS
            ProductVersionLS = pe.VS_FIXEDFILEINFO.ProductVersionLS
            ProductVersionMS = pe.VS_FIXEDFILEINFO.ProductVersionMS
        except Exception as e:
            result=["error"]
        #print "{} while opening {}".format(e,filepath)
        else:
    #shifting byte
            FileVersion = (FileVersionMS >> 16, FileVersionMS & 0xFFFF, FileVersionLS >> 16, FileVersionLS & 0xFFFF)
            ProductVersion = (ProductVersionMS >> 16, ProductVersionMS & 0xFFFF, ProductVersionLS >> 16, ProductVersionLS & 0xFFFF)
            result = [FileVersion,ProductVersion,ProductName,CompanyName]
        return int ( result[0] != 'error')

    def write_csv_header(self):
        if os.path.exists(self.output): return
        filepath = self.output
        header= self.IMAGE_DOS_HEADER + self.FILE_HEADER + self.OPTIONAL_HEADER + self.Derived_header
        header.append("class")
        csv_file= open(filepath,"a")
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(header)
        csv_file.close()

    def extract_all(self,filepath):
        data =[]


        os.system(f'cp {filepath} /tmp/1.exe')
        filepath = '/tmp/1.exe'

        try:
            pe = pefile.PE(filepath)
        except Exception as e:
            print(e)
            pass
            
        try:
            data += self.extract_dos_header(pe)
        except Exception as err:
            print('ERROR EXTRACT DOS HEADER: ' + str(err))
            return None

        try:
            data += self.extract_file_header(pe)
        except Exception as err:
            print('ERROR EXTRACT FILE HEADER: ' + str(err))
            return None

        try:
            data += self.extract_optional_header(pe)
        except Exception as err:
            print('ERROR EXTRACT OPTIONAL HEADER: ' + str(err))
            return None
            
            # derived features
            #number of suspicisou sections and non-suspicsious section
        try:
            num_ss_nss = self.get_count_suspicious_sections(pe)
            data += num_ss_nss
        except Exception as err:
            print('ERROR get_count_suspicious_sections: ' + str(err))
            return None

        # try:
        #     packer = self.check_packer(filepath)
        #     data += packer[0]
        # except Exception as err:
        #     print('ERROR check packer: ' + str(err))
        #     return None

        try:
            entropy_sections = self.get_text_data_entropy(pe)
            data += entropy_sections
        except Exception as err:
            print('ERROR check entropy: ' + str(err))
            return None

        try:
            f_size_entropy = self.get_file_entropy(filepath)
            data += f_size_entropy
        except Exception as err:
            print('ERROR check fsize entropy: ' + str(err))
            return None

        try:
            fileinfo = self.get_fileinfo(pe)
            data.append(fileinfo)
            data.append(self.type)
        except Exception as err:
            print('ERROR check fileinfo: ' + str(err))
            return None
            
        return data
    
    def write_csv_data(self,data):
        filepath = self.output
        csv_file= open(filepath,"a")
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(data)
        csv_file.close()
    

    def create_dataset(self):
        self.write_csv_header()
        #run through all file of source and extract features

        count = 0
        files = os.listdir(self.source)
        lenfile = len(files)
        for file in files:         
                print("Extract: {} / {}".format(count, lenfile))
                count += 1
                filepath = self.source + file
                data = self.extract_all(filepath)
                print(data)
                if data != None and len(data)>0:
                    self.write_csv_data(data)
                    print("Successfully Data extracted and written for {}.".format(file))

In [6]:
PATH_BENIGN = './Dataset/files/benign/'
PATH_MALWARE = './Dataset/files/malware/'
PATH_BENIGN = '/mnt/hgfs/dataset/'

import os

PE_EXTRACTOR = pe_features(source=PATH_BENIGN, output='new.csv', label="0")
PE_EXTRACTOR.create_dataset()

# PE_EXTRACTOR = pe_features(source=PATH_MALWARE, output='dataset.csv', label="1")
# PE_EXTRACTOR.create_dataset()


Extract: 0 / 2334
[144, 3, 4, 65535, 184, 256, 5, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 14, 16, 81408, 27136, 0, 30941, 4096, 86016, 1, 1, 1, 5, 1, 0, 0, 5, 1, 1, 1, 164418, 2, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1048576, 4096, 1048576, 4096, 1, 5, 0, 0.0, 0.0, 116880, 6.524627405101686, 0, '0']
Successfully Data extracted and written for 001405CE2F3363F6C22585D152A14969.exe.
Extract: 1 / 2334


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f78fc78f580>>
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 